In [5]:
# https://www.tensorflow.org/lite/tutorials/model_maker_image_classification
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from TarfileFunctions import *

from time import perf_counter, sleep
import itertools, json, os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

import numpy as np
import cv2
import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

# from tflite_support.metadata_writers import image_classifier
# from tflite_support.metadata_writers import writer_utils
import matplotlib.pyplot as plt

contentPath=os.getcwd()
image_path=os.path.join(contentPath, 'images')
generatorPath=os.path.join(contentPath, 'DataGenerator')
testPath=join(contentPath, 'images')
imagePath=join(contentPath, '3b7d7d8a64.jpg')

initialGlobList:list
with open("initialGlobList.json", 'r') as f:
    initialGlobList = json.load(f)
    
def listNewFiles(initial=initialGlobList, delete=False):
    currentFilesGlob=glob.glob('**')
    if len(initial) == len(currentFilesGlob):
        print(f'{C.BIRed}No new files in content.')
    for fil in currentFilesGlob:
        if not fil in initial:
            if isdir(fil):
                print(f'{C.BIBlue}{fil}')
                if delete:
                    shutil.rmtree(fil)
            elif isfile(fil):
                print(f'{C.ColorOff}{fil}')
                if delete:
                    os.remove(fil)
listNewFiles()

No new files in content.


In [6]:
# Load input data specific to an on-device ML app.
data = DataLoader.from_folder(generatorPath)
train_data, restData = data.split(0.9)
print(train_data.size//32)
validation_data, test_data = restData.split(0.5)

INFO:tensorflow:Load image with size: 844, num_label: 3, labels: Enemy, Friendly, Planets.
23


In [14]:
# Load input data specific to an on-device ML app.
testData = DataLoader.from_folder(testPath)
print(testData.index_to_label)
# validation_data, test_data = restData.split(0.5)

INFO:tensorflow:Load image with size: 4, num_label: 1, labels: TestImages.
['TestImages']


In [ ]:
# help(image_classifier.create)

In [ ]:
start=perf_counter()
# Customize the TensorFlow model.
model = image_classifier.create(
    train_data,
    model_spec=model_spec.get('mobilenet_v2'),
    epochs=1,
    dropout_rate=0.25,
    learning_rate=0.001,
    momentum=0.9)

finish=perf_counter()
cvu.printTime(start, finish)

In [ ]:
# accuracy: 90.0%
# Evaluate the model.
loss, accuracy = model.evaluate(testData)

loss = round(loss, 4)
accuracy = round(accuracy, 4)
print(f'loss: {loss}\taccuracy: {accuracy * 100}%')


In [ ]:
length=len(test_ds)
def get_label_color(val1, val2):
  if val1 == val2:
    return 'green'
  else:
    return 'red'
num=len(test_ds)
plt.figure(figsize=(224, 224))
plt.rcParams.update({'font.size': 10})
predicts = model.predict_top_k(test_ds)
for i, (image, label) in enumerate(test_ds.gen_dataset().unbatch().take(num)):
    ax = plt.subplot(num, 1, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image.numpy(), cmap=plt.cm.gray) # .gray

    predict_label = predicts[i][0][0]
    color = get_label_color(predict_label,
                          test_ds.index_to_label[label.numpy()])
    ax.xaxis.label.set_color(color)
    plt.xlabel('Predicted: %s' % predict_label)
plt.show()